In [ ]:
from ctypes import cdll, c_double, POINTER, c_int
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# load the shared library
lib = cdll.LoadLibrary("./libprizmo.so")

In [ ]:
# define the init function
lib.prizmo_init_c.argtypes = None
lib.prizmo_init_c.restype = None
lib.prizmo_init_c()

In [ ]:
#  define prizmo_evolve_c(x, Tgas, jflux, dt, verboseChem, errState)
lib.prizmo_evolve_c.argtypes = [POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_int), POINTER(c_int)]
lib.prizmo_evolve_c.restype = None

# wrapper function for evolve
def evolve(x, Tgas, jflux, dt):
    x = (c_double * len(x))(*x)
    jflux = (c_double * len(jflux))(*jflux)

    tgas = c_double(Tgas)

    ierr = 0
    verboseChem = 0

    lib.prizmo_evolve_c(
        x,
        tgas,
        jflux,
        c_double(dt),
        c_int(verboseChem),
        c_int(ierr)
    )

    return np.array(list(x)), tgas.value

In [ ]:
# hardcoded values from prizmo_commons.f90
nspecies = 33
nphoto = 1000

# hardcoded values from prizmo_commons.f90 (note F90 arrays are 1-indexed)
idx_Cj = 13
idx_E = 14
idx_H2 = 16
idx_O = 27

seconds_per_year = 365. * 24 * 3600

# abundances in cm^-3
x = np.zeros(nspecies)
x[idx_Cj] = 1e-4  # C+
x[idx_E] = 1e-4   # e-
x[idx_H2] = 1e0  # H2
x[idx_O] = 2e-4   # O

# scale to total number density of 1e4 cm^-3
x *= 1e4

Tgas = 1e2 # gas temperature in K
jflux = np.zeros(nphoto) + 1e-40  # negligible radiation field

dt = 1e-1 * seconds_per_year  # initial time, s
tend = 1e6 * seconds_per_year # end time, s

# arrays to store results
all_x_Cj = []
all_x_E = []
all_tgas = []
tt = []

# initial time, s
t = 0e0

while True:
    dt = min(dt*1.3, tend)
    x, Tgas = evolve(x, Tgas, jflux, dt)

    t += dt

    all_x_Cj.append(x[idx_Cj])
    all_x_E.append(x[idx_E])
    all_tgas.append(Tgas)
    tt.append(t)

    if t >= tend:
        break


# Convert lists to numpy arrays for easier handling
all_x_Cj = np.array(all_x_Cj)
all_x_E = np.array(all_x_E)
all_tgas = np.array(all_tgas)
tt = np.array(tt)

In [ ]:
plt.plot(tt / seconds_per_year, all_x_Cj, label='C+')
plt.plot(tt / seconds_per_year, all_x_E, label='e-')
plt.plot(tt / seconds_per_year, all_tgas, label='Tgas')
plt.xlabel('Time (s)')
plt.ylabel('Abundance and Temperature')
plt.legend()
plt.xscale('log')
plt.yscale('log')


In [ ]:
#(x, Tgas, Tdust, jflux, cools)
lib.prizmo_get_cooling_array_c.argtypes = [POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double)]
lib.prizmo_get_cooling_array_c.restype = None

def get_cooling_array(x, Tgas, Tdust, jflux):
    x = (c_double * len(x))(*x)  # Convert to a C array
    jflux = (c_double * len(jflux))(*jflux)  # Convert to a C array

    cools = [0.0] * 5  # Initialize a list for cooling values
    cools = (c_double * len(cools))(*cools)  # Create an array to hold cooling values

    lib.prizmo_get_cooling_array_c(
        x,
        c_double(Tgas),
        c_double(Tdust),
        jflux,
        cools
    )

    return np.array(list(cools))  # Convert to a Python list